In [74]:
import os
import time
from tqdm import tqdm
from PIL import Image
from pdf2image import convert_from_path,convert_from_bytes
import tempfile
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

class pdfToImage:
    def __init__(self,pdfname, imagePath, *pageNum):
        self.pdfname=pdfname
        self.imagePath=imagePath
        self.pagenum=pageNum
        
    def pdf2image(self):
        pgns=self.outpageNum()

        
        with tempfile.TemporaryDirectory() as path:
            print("正在读取Pdf文件…… \n")
            images_from_path = convert_from_path(self.pdfname, output_folder=path, dpi=96)
            print("文件总页数 {} 页。 \n".format(len(images_from_path)))
            print("正在转换…… \n")   
            time.sleep(0.5)
            for pgn in tqdm(pgns):
                if not os.path.exists(self.imagePath):
                    os.makedirs(self.imagePath)
                images_from_path[int(pgn)].save(self.imagePath+'/'+'psReport_%s.jpg' % str(int(pgn+1)).zfill(3),'jpeg')
#             print(images_from_path)
        print("done")
    
    def image_adj(self,w,h):
        
        print("正在调整图片大小至{0}x{1}... \n".format(w,h))
        for pic in os.listdir(self.imagePath):            
            if pic[-3:].lower()=="jpg":
                _img=Image.open(os.path.join(self.imagePath,pic))
                img=_img.resize((w,h))
                img.save(os.path.join(self.imagePath,pic))
        print("done \n")
                                                

    def outpageNum(self):
        pgn=[]
        
        if isinstance(self.pagenum[0],str):
            if "-" in self.pagenum[0]:
                firstpage=self.pagenum[0][:self.pagenum[0].find("-")]
                lastpage=self.pagenum[0][self.pagenum[0].find("-")+1:]
                for i in range(int(firstpage)-1,int(lastpage)-1):
                    pgn.append(i)
                pgn.append(int(lastpage)-1) #包括最后一页
            else:
                pass
        else:
            pgn=[int(x)-1 for x in self.pagenum]

        return pgn
        
    
if __name__=="__main__":
    pdf='//home//jack//文档//大智小超//乐高教育.pdf'
    jpg='//home//jack//文档//大智小超/output_pdf'
    
    pdf=pdfToImage(pdf,jpg,7,9,12,18)
#     pdf.pdf2image()
    pdf.image_adj(1024,576)
    

正在调整图片大小至1024x576... 

done 



In [68]:
import time
from tqdm import tqdm
# from tqdm.std import trange
 
a=[3,8,9,12]
for i in tqdm(a):
    time.sleep(0.5)

100%|██████████| 4/4 [00:02<00:00,  1.99it/s]
